## **Task-1: Design a input scheme for the user to input a chemical equation that need to be balanced.**

### **Importing required modules**

In [108]:
import re
import numpy.linalg as la
%pylab inline
from fractions import Fraction

Populating the interactive namespace from numpy and matplotlib


### **Input, parsing input**

In [109]:
'''taking a string of equation to process and balance in the program'''
string="H 2 + O 2 = H 2 O"

In [110]:
'''using regex to parse the string into a list of atoms'''
atomList = set(re.split('[ 0-9+=]+',string))
print(atomList)

{'O', 'H'}


In [111]:
'''split the string into reactants and products by splitting across the ='''
[reactants, products] = (re.split('=',string))

In [112]:
'''split the reactant and product modules by splitting across the +'''
react=reactants.split(" + ")
prod=products.split(" + ")

## **TASK 1A: Write a function that will take one chemical (reactant or product) and return `chemical_composition_vector`**


In [113]:

def chemical_composition_vector( input, atomList ):
    ''' Returns a vector with the chemical composition of the supplied molecule'''
    out = [0]*len(atomList)
    c=list((re.split(' ',input)))

    # Iterate len(c) times
    for i in range(len(c)):
        # Check whether the the character is a space or a digit, if yes, continue
        if(c[i].isdigit() or c[i]==" " ):
            continue
        # in all the other cases
        else:
            # initialise the count variable to 0 before the inner loop
            j = 0
            # iterate through each chacter in the atomlist
            for z in atomList:
                # if the character in atomlist is the same as the current atom in molecule
                if( z == c[i] ):
                    # stop iterating
                    break
                else:
                    # if the character is not hte same, continue to iterate and increment count variable
                    j += 1
            # check if this is the last character in i + 1
            if( (i+1)<len(c) ):
                #check if the next character is a number
                if c[i+1].isdigit() :
                    # make output's jth character this number, i.e. coefficient
                    out[j] = int(c[i+1])
            # if the atom only occurs once, make the coefficient = 1
            else:
                out[j] = 1
    # finally return 'out'
    return out

In [114]:
'''testing the above function'''
print("Chemical compositions of the reactants and products are:")
# for reactant
r1=chemical_composition_vector("H 2",atomList)
print("r1 =", r1)
r2=chemical_composition_vector("O 2",atomList)
print("r1 =", r2)
# for product
p1=chemical_composition_vector("H 2 O",atomList)
p1=[-h for h in p1]
print("p1 =", p1)

Chemical compositions of the reactants and products are:
r1 = [0, 2]
r1 = [2, 0]
p1 = [-1, -2]


## **Task-2: From the input, get into a form that is suitable for mathematical manipulations**

In [115]:
'''appending 0 to all the lists'''
r1.append(0)
r2.append(0)
p1.append(0)
#3*3 matrix for r1,r2,p1 
A=np.array([r1,r2,p1])
print(A)

[[ 0  2  0]
 [ 2  0  0]
 [-1 -2  0]]


In [116]:
'''transpose the matrix'''
A_T = numpy.transpose(A)
print(A_T)

[[ 0  2 -1]
 [ 2  0 -2]
 [ 0  0  0]]


In [117]:
'''compute the eigenvalues and eigenvalue'''
a_eig_val,A_eig=la.eig(A_T)
print(A_eig)

[[ 0.70710678 -0.70710678  0.66666667]
 [ 0.70710678  0.70710678  0.33333333]
 [ 0.          0.          0.66666667]]


In [118]:
'''a1,a2,a3 are the coefficients of H2,O2 and H2O'''
a1=A_eig[0,2]
a2=A_eig[1,2]
a3=A_eig[2,2]

In [119]:
'''not to be used unless testing, or making the coefficients whole'''
a_min=min(a1,a2,a3)
# a1=a1/a_min
# a2=a2/a_min
# a3=a3/a_min
'''printing the above obtained coefficients'''
print("coeff of H2 = ", a1)
print("coeff of O2 = ", a2)
print("coeff of H2O = ", a3)

coeff of H2 =  0.6666666666666667
coeff of O2 =  0.3333333333333331
coeff of H2O =  0.6666666666666666


## **Task-3: Using the solution above, output the balanced equation**

In [120]:
'''print eqn with coefficient'''
print(a1,react[0],'+',a2,react[1],'=',a3,products)

0.6666666666666667 H 2 + 0.3333333333333331 O 2  = 0.6666666666666666  H 2 O


## **Task-4: If the above is working, can you wrap the whole program in a single function, which will take the input and give the proper output?**

In [123]:
'''PUTTING THE ABOVE IN A GENERALISED NON HARDCODED FUNCTION'''
def setAtomList(string):
    global atomList
    '''using regex to parse the string into a list of atoms'''
    atomList = set( re.compile('[A-z]+').findall(string) )
    atomList = list(atomList)
    atomList = sorted(atomList)

def BalanceChemicalEquation(input_string):
    '''Balances the input equation with float coefficients'''

    #parsing and dividing input
    equation = input_string
    setAtomList(equation)
    '''splits the string across left and right side, so split_eq[0] contains all the reactants, split_eq[1] contains all the products'''
    split_eq = list(re.split(' = ', equation))
    '''get a list of all the reactants in a list'''
    reactants = list(re.split(' \++ ', split_eq[0]))
    '''get a list of all the products in a list'''
    products = list(re.split(' \++ ', split_eq[1]))

    #number and length calculation
    number_of_mols = len(reactants) + len(products)
    no_of_atom_species = len(atomList)
    size_of_array = max(number_of_mols, no_of_atom_species)

    #populate the resultant_matrix with 0's
    resultant_matrix = np.zeros([size_of_array,size_of_array])
    #initialise the count variable to 0 before the inner loop
    remaining = 0

    
    #iterating through the number of reactants
    for i in range(len(reactants)):
        #
        res = chemical_composition_vector( reactants[i], atomList )
        #double looping through res
        for j in range(len(res)):
            resultant_matrix[i][j] = res[j]
        remaining = i

    remaining += 1
    #iterating through the number of products
    for i in range(len(products)):
        res = chemical_composition_vector( products[i], atomList )
        for j in range(len(res)):
            resultant_matrix[i+remaining][j] = -res[j]
    resultant_matrix = np.transpose(resultant_matrix)
    #here resultant[0] is the values, [1] = vectors
    resultant = la.eig(resultant_matrix)

    eigenvalues = resultant[0]

    for i in range(len(eigenvalues)):
        eigenvalues[i] = np.round(eigenvalues[i],decimals = 3)

    eigenvectors = resultant[1]

    for i in range(len(eigenvalues)):
        if eigenvalues[i] == 0:
            dec_coeff = np.transpose(eigenvectors)[i]

    minim = min(dec_coeff)
    dec_coeff = dec_coeff/minim

    for i in range(len(dec_coeff)):
        dec_coeff[i] = np.round(dec_coeff[i],decimals = 3)
    
    print("\nBalanced equation: ")
    balanced_eqn=""
    if len(reactants) == 1:
        balanced_eqn = str(balanced_eqn) + str(dec_coeff[0]) + str(reactants[-1]) + " = "
        j = 1
    else:
        for i in range(len(reactants)-1):
            balanced_eqn = str(balanced_eqn) + str(dec_coeff[i]) + str(reactants[i]) + " + " + str(dec_coeff[i+1]) + str(reactants[-1]) + " = "
        j = i+2

    if len(products) == 1:
        balanced_eqn = str(balanced_eqn) + str(dec_coeff[j]) + str(products[-1])
    else:
        for i in range(len(products)-1):
            balanced_eqn = str(balanced_eqn) + str(dec_coeff[i+j]) + str(products[i]) + " + " + str(dec_coeff[i+j+1]) + str(products[-1])
    print(balanced_eqn)

## **Using the above for equations**
**NOTE:** The above function has NOT been hardcoded so, it works for more than 2 reactants and 1 product equations.

## **Example 1:**
```py
H 2 + Cl 2 = H 1 Cl 1
```


In [124]:
eqn = "H 2 + O 2 = H 2 O"
BalanceChemicalEquation(eqn)


Balanced equation: 
2.0H 2 + 1.0O 2 = 2.0H 2 O
